In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from PIL import Image
import os

input_folder = 'NLM Filters/test/severe_dysplastic_filtered_nlm'
output_folder = 'NLM Augmentation/test/severe_dysplastic_filtered_nlm_aug'

# Create an ImageDataGenerator and specify the augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    rescale=1./255  # Normalize pixel values
)

# Generate augmented images and save them to the output folder
for filename in os.listdir(input_folder):
    image_path = os.path.join(input_folder, filename)

    # Load the image using Keras' load_img function
    image = load_img(image_path)

    # Resize the image to your desired size (e.g., 224x224)
    target_size = (224, 224)
    image = image.resize(target_size)

    # Convert the image to a NumPy array
    image_array = img_to_array(image)

    # Reshape the image array to match the expected input shape of the generator
    image_array = image_array.reshape((1,) + image_array.shape)

    # Generate augmented images using the datagen.flow() method
    augmented_images = datagen.flow(
        image_array,
        batch_size=16,
        save_to_dir=output_folder,
        save_prefix='augmented',
        save_format='png'
    )

    # Generate and save the augmented images
    num_augmented_images = 5
    for i, augmented_image in enumerate(augmented_images):
        if i >= num_augmented_images:
            break

        augmented_image_pil = Image.fromarray((augmented_image[0] * 255).astype('uint8'))  # Ensure correct range

        save_filename = f'{filename.split(".")[0]}_{i}.png'
        save_path = os.path.join(output_folder, save_filename)
        augmented_image_pil.save(save_path)

print("Augmented images done ")


Augmented images done 
